Notebook responsável para realizar transformações no dataframe de contratos. Ele está sendo usado apenas para testes, código completos serão enviados para o arquivo de transform.py

Carregamento do arquivo bruto de contratos 👇

In [1]:
from pathlib import Path
import pandas as pd

# Configura o caminho para os arquivos brutos
raw_data_dir = Path('..') / 'data' / 'raw'

# Localiza o arquivo de contratos mais recente (pelo formato de data no nome do arquivo)
contratos_files = list(raw_data_dir.glob('contratos_amostra_*.csv'))

if contratos_files:
    # Pega o arquivo mais recente baseado na data de modificação
    latest_contratos_file = max(contratos_files, key=lambda x: x.stat().st_mtime)

    # Verificação (para debug)
    print(f'Arquivo de contratos encontrado: {latest_contratos_file}')
    print(f'O arquivo existe? {latest_contratos_file.exists()}')

    # Carrega o Dataframe
    df = pd.read_csv(latest_contratos_file, encoding='utf-8', low_memory=False)

    # Visualiza dimensões e primeiras linhas
    print(f'\nDataframe carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')
    print(f'Primeira 5 colunas: {', '.join(df.columns[:5])}')

else:
    print('ERRO: Nenhum arquivo de contratos encontrado em data/raw')
    print(f'Diretório verificado: {raw_data_dir.resolver()}')
    print(f'O diretório existe? {raw_data_dir.exists()}')
    print('Arquivos no diretório:', list(raw_data_dir.glob('*')))

Arquivo de contratos encontrado: ..\data\raw\contratos_amostra_2025-08-03.csv
O arquivo existe? True

Dataframe carregado: 12000 linhas, 39 colunas
Primeira 5 colunas: codigoOrgao, nomeOrgao, codigoUnidadeGestora, nomeUnidadeGestora, codigoUnidadeGestoraOrigemContrato


In [2]:
df.head(10)

,codigoOrgao,nomeOrgao,codigoUnidadeGestora,nomeUnidadeGestora,codigoUnidadeGestoraOrigemContrato,nomeUnidadeGestoraOrigemContrato,receitaDespesa,numeroContrato,codigoUnidadeRealizadoraCompra,nomeUnidadeRealizadoraCompra,...,valorAcumulado,totalDespesasAcessorias,dataHoraInclusao,numeroControlePncpContrato,idCompra,dataHoraExclusao,contratoExcluido,unidadesRequisitantes,data_inicio,trimestre
0,52121,ESCRITÓRIO AVANÇADO DA OPERAÇÃO CARRO-PIPA DA ...,160454,28º BATALHAO DE CACADORES,160454,28º BATALHAO DE CACADORES,D,00003/2024,160454.0,28º BATALHAO DE CACADORES,...,57606.44,NaN,2024-02-20T14:46:03,NaN,16045407000012024,NaN,False,NaN,2024-01-01,1
1,26407,"INST.FED.DE EDUC.,CIENC.E TEC.GOIANO",158124,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",158124,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",D,2024NE000042,158124.0,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",...,NaN,NaN,2024-07-01T10:33:34,NaN,15812407000032023,NaN,False,NaN,2024-01-26,1
2,5604,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,925068,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,925068,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,D,2024NE038305,925068.0,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,...,631.58,NaN,2024-07-24T12:01:39,NaN,92506806900072024,NaN,False,NaN,2024-03-18,1
3,26430,"INST.FED.DE ED.,CIENC.E TEC.DO S.PERNAMBUCANO",158499,INST.FED.SERTAO PERNAMBUCANO/CAMPUS PETROLINA,158499,INST.FED.SERTAO PERNAMBUCANO/CAMPUS PETROLINA,D,00010/2024,158276.0,INST.FED.DO MARANHAO/CAMPUS SAO LUIS-MACARANA,...,NaN,NaN,2024-07-01T12:00:59,NaN,15827605000052023,NaN,False,NaN,2024-03-25,1
4,38576,CONSELHO REG. DE ENGENHARIA E AGRONOMIA-SC,389087,CONSELHO REGIONAL DE ENGENHARIA E AGRONOMIA DE...,389087,CONSELHO REGIONAL DE ENGENHARIA E AGRONOMIA DE...,D,2024NE000567,389087.0,CONSELHO REGIONAL DE ENGENHARIA E AGRONOMIA DE...,...,1909.00,NaN,2024-07-15T16:51:09,NaN,38908706000092024,NaN,False,NaN,2024-02-16,1
5,52121,ESCRITÓRIO AVANÇADO DA OPERAÇÃO CARRO-PIPA DA ...,160496,ER OP C PIPA/6ª RM,160496,ER OP C PIPA/6ª RM,D,00086/2024,160496.0,ER OP C PIPA/6ª RM,...,NaN,NaN,2024-08-08T10:09:50,NaN,16049607000122023,NaN,False,NaN,2024-01-01,1
6,36000,MINISTERIO DA SAUDE,250057,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,250057,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,D,2024NE000018,250057.0,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,...,891464.00,NaN,2024-07-09T16:57:35,NaN,25005705001572023,NaN,False,NaN,2024-01-08,1
7,36000,MINISTERIO DA SAUDE,250057,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,250057,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,D,2024NE000016,250057.0,INST. NACIONAL DE TRAUMATOLOGIA E ORTOPEDIA,...,55000.00,NaN,2024-07-09T17:54:33,NaN,25005705001662023,NaN,False,NaN,2024-01-08,1
8,52121,ESCRITÓRIO AVANÇADO DA OPERAÇÃO CARRO-PIPA DA ...,160454,28º BATALHAO DE CACADORES,160454,28º BATALHAO DE CACADORES,D,00108/2024,160454.0,28º BATALHAO DE CACADORES,...,NaN,NaN,2024-03-20T14:37:09,NaN,16045407000012024,NaN,False,NaN,2024-01-01,1
9,52121,ESCRITÓRIO AVANÇADO DA OPERAÇÃO CARRO-PIPA DA ...,160552,ESCRITORIO REGIONAL OP C PIPA/7ª RM,160552,ESCRITORIO REGIONAL OP C PIPA/7ª RM,D,00303/2023,160552.0,ESCRITORIO REGIONAL OP C PIPA/7ª RM,...,65634.24,NaN,2023-12-15T18:09:13,NaN,16055207900052023,NaN,False,NaN,2024-01-01,1


Verificação de Nulos por quantidade e porcentagem 👇

In [4]:
# Calcula contagem e percentual de nulos em cada coluna
nulos = df.isnull().sum()
percentual = (df.isnull().sum() / len(df) * 100).round(2)

# Cria um Dataframe simples apenas com colunas que têm pelo menos um valor nulo
colunas_com_nulos = pd.DataFrame({
    'Valores Nulos': nulos[nulos >0],
    'Percentual (%)': percentual[nulos > 0]
}).sort_values('Percentual (%)', ascending=False)

# Exibe o resultado
colunas_com_nulos

,Valores Nulos,Percentual (%)
numeroControlePncpContrato,12000,100.00
dataHoraExclusao,12000,100.00
totalDespesasAcessorias,11999,99.99
nomeSubcategoria,11462,95.52
codigoSubcategoria,11462,95.52
unidadesRequisitantes,10824,90.20
informacoesComplementares,8196,68.30
valorAcumulado,6287,52.39
codigoTipo,4997,41.64
nomeTipo,4997,41.64


MANTER PARA CASO SEJA NECESSÁRIO MAIS INFORMAÇÕES SOBRE ALGUM REGISTRO (MESMO COM BASTANTE NULOS)
- `informacoesComplementares`

EM DÚVIDA SE DEVO MANTER OU NÃO
- `valorAcumulado` -> dá para descobrir alguns nulos baseado nas colunas 'valorGlobal', 'numeroParcelas' e 'valorParcela' (manter cuidado ao fazer isso para evitar viés de dados -> olhar melhor depois)
- `codigoTipo` e `nomeTipo` -> analisar se convém manter ou excluir (levando em conta que mais de 90% dos registros não nulos são empenho)

POUCOS NULOS (manter eles e adicionar flags)
- `numeroCompra` -> é provável que não dê para repor os nulos, ver o que fazer
- `codigoUnidadeRealizadoraCompra` e `nomeUnidadeRealizadoraCompra` -> é provável que não dê para repor nulos, mesmo vendo pelo número de contrato, continua confuso
- `dataVigenciaFinal` -> colocar flags para sinalizar os nulos
- `nomeCategoria` ->
- `codigoCategoria` ->
- `nomeRazaoSocialFornecedor` -> dá para descobrir os nulos baseado no 'niFornecedor'

### ANÁLISE DAS COLUNAS NULAS


**Análise da coluna `nomeSubcategoria`** \
*OBS: também inclui `codigoSubcategoria`*
- Total de nulos: 95,52%
- Quantidade de nulos baseado na coluna `nomeCategoria`: maioria acima de 95%
- Valores não nulos mais comuns: Nota Fiscal Eletrônica, Quinzenal, Diária, etc
- Conclusão: coluna com altíssima proporção de nulos e baixa utilidade -> <span style='color:red; font-weight:bold;'>descartar</span>


In [19]:
import re
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeSubcategoria' in df.columns:
    # Verifica porcentagem de nulos de 'nomeSubcategoria'
    nulos_sub = df['nomeSubcategoria'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_sub / total * 100).round(2)

    print("Relação entre 'nomeSubcategoria' e outras colunas:")
    print(f"\nTotal de nulos: {nulos_sub} ({percentual_nulos}%)")

    # 1. Verifica relação com coluna 'nomeCategoria'
    if 'nomeCategoria' in df.columns:
        nulos_por_categoria = df.groupby('nomeCategoria')['nomeSubcategoria'].apply(lambda x: x.isnull().mean() * 100).round(2)
        print("\nPercentual de nulos em 'nomeSubcategoria' por 'nomeCategoria':")
        print(nulos_por_categoria.sort_values(ascending=False))

    # 2. Valores não nulos mais comuns
    # Limpeza de valores não nulos (remove URLs)
    df['nomeSubcategoria'] = df['nomeSubcategoria'].apply(lambda x: x if pd.isnull(x) or not re.match(r'^https?://', str(x)) else None)

    if df['nomeSubcategoria'].notnull().any():
        valores_comuns = df['nomeSubcategoria'].value_counts().head(10)
        print("\nValores mais comuns que aparecem em 'nomeSubcategoria':")
        print(valores_comuns)

    # 3. Conclusão baseada na análise
    print("\nConclusão baseada na análise:")
    if percentual_nulos > LIMITE_NULOS:
        print(f"- Mais de {LIMITE_NULOS}% de nulos → candidata forte à exclusão")
    else:
        print("- Percentual de nulos abaixo do limite, avaliar manutenção")

Relação entre 'nomeSubcategoria' e outras colunas:

Total de nulos: 11562 (96.35%)

Percentual de nulos em 'nomeSubcategoria' por 'nomeCategoria':
nomeCategoria
Serviços de Saúde         99.53
Serviços                  98.84
Serviços de Engenharia    97.60
Informática (TIC)         95.40
Compras                   95.13
Obras                     95.06
Cessão                    93.24
Mão de Obra               85.07
Name: nomeSubcategoria, dtype: float64

Valores mais comuns que aparecem em 'nomeSubcategoria':
nomeSubcategoria
Nota Fiscal Eletrônica                        111
Quinzenal                                     105
Diária                                         66
Assistente Social                              27
Semanal                                        22
Produtor rural polivalente na agricultura       7
Operador de acabamento (indústria gráfica)      6
CONDOMÍNIO EDILÍCIO                             5
Trabalhador agropecuário em geral               4
Embalagem 250 Grama 

---

**Análise da coluna `unidadesRequisitantes`**
- Total de nulos: 90,20%
- Quantidade de nulos baseado na coluna : 
- Valores não nulos mais comuns: 
- Conclusão: a -> <span style='color:white; font-weight:bold;'>a</span>

In [ ]:
pass

---

Exclusão de algumas colunas 👇

In [ ]:
colunas_para_excluir = [
    'numeroControlePncpContrato',
    'dataHoraExclusao',
    'totalDespesasAcessorias',
    'nomeSubcategoria',
    'codigoSubcategoria',

    'unidadesRequisitantes',
    
    'contratoExcluido', # não é uma coluna de interesse
    ]

df.drop(colunas_para_excluir, axis=1, inplace=True)

In [6]:
len(df.columns)

39

In [5]:
# Verifica se há relação com o tipo de contrato
if 'nomeTipo' in df.columns:
    # Calcular o percentual de nulos para cada grupo
    nulos_por_tipo = df.groupby('nomeTipo')['numeroCompra'].apply(
        lambda x: (x.isnull().sum() / len(x) * 100).round(2)
    )
    print("\nPercentual de numeroCompra nulo por tipo de contrato:")
    print(nulos_por_tipo.sort_values(ascending=False).head())


Percentual de numeroCompra nulo por tipo de contrato:
nomeTipo
Acordo de Cooperação Técnica (ACT)    72.48
Termo de Compromisso                  57.14
Outros                                 9.40
Empenho                                0.56
Carta Contrato                         0.00
Name: numeroCompra, dtype: float64


- termo de compromisso (4)-> numeroCompra não se aplica, pois são doações
- 